In [1]:
%config IPCompleter.use_jedi=False

# Tweepy experiments

In [29]:
import tweepy, json

class BaseAgent():
    def __init__(self, credentials, wait_on_rate_limit=True):
        auth = tweepy.OAuth1UserHandler(
            credentials["api_key"], credentials["api_secret"], credentials["access_token"], credentials["access_token_secret"]
        )
        self.api = tweepy.API(auth, wait_on_rate_limit=True, tweet_mode='')
        
class QueryUserConnections(BaseAgent):
    def __init__(self, credentials, query_friends=True, wait_on_rate_limit=True):
        super(QueryUserConnections, self).__init__(credentials, wait_on_rate_limit)
        self.query_friends = query_friends
        
    def execute(self, screen_name):
        if self.query_friends:
            results = tweepy.Cursor(self.api.get_friend_ids, screen_name=screen_name).items()
        else:
            results = tweepy.Cursor(self.api.get_follower_ids, screen_name=screen_name).items()
        user_ids = []
        while True:
            try:
                user_id = next(results)
            except tweepy.errors.TooManyRequests:
                time.sleep(60*15)
                user_id = next(results)
            except StopIteration:
                break
            #print(user_id)
            user_ids.append(user_id)
        return user_ids
        
class LookupUserIds(BaseAgent):
    def __init__(self, credentials, wait_on_rate_limit=True):
        super(LookupUserIds, self).__init__(credentials, wait_on_rate_limit)
        self.queue = []
        
    def add(self, queries, high_priority=False):
        if high_priority:
            self.queue = queries + self.queue
        else:
            self.queue += queries
            
    @property
    def is_long(self):
        return len(self.queue) > 100
    
    def __len__(self):
        return len(self.queue)
            
    #TODO: make magic for exceptions...
    def _make_query(self):
        if self.is_long:
            queries = self.queue[:100].copy()
            self.queue = self.queue[100:]
        else:
            queries = self.queue.copy()
            self.queue = []
        queries = list(set(queries))
        users = self.api.lookup_users(user_id=queries)
        return users
            
    def execute(self, force=False):
        users = []
        if force:
            while len(self.queue) > 0:
                users += self._make_query()
        else:
            while self.is_long:
                users += self._make_query()
        return users

In [30]:
with open("api_key.json") as f:
    credentials = json.load(f)

In [31]:
friend_agent = QueryUserConnections(credentials)
follow_agent = QueryUserConnections(credentials, query_friends=False)
user_agent = LookupUserIds(credentials)

TypeError: __init__() got an unexpected keyword argument 'extended'

In [5]:
friend_ids = friend_agent.execute("ferencberes91")
len(friend_ids)

126

In [6]:
user_agent.add(friend_ids)

In [7]:
follow_ids = follow_agent.execute("ferencberes91")
len(follow_ids)

55

In [8]:
user_agent.add(follow_ids)

In [9]:
len(user_agent)

181

In [10]:
users = user_agent.execute(force=True)
len(users)

176

In [11]:
users[0].screen_name

'paulmorio'

#### Notes: környezet felderítése
- friend-eket mindig lekérjük - gyorsan megy (jellemzi a user-t)
- followerek: itt csak azokat kérjük le akik az adatban több fiókot is követnek - (pl. MAC esetén nem fogunk 500K fiókot lekérdezni...)

In [48]:
api = friend_agent.api

In [53]:
query = {
    "screen_name":"ferencberes91", 
    "tweet_mode":'extended'
}

In [54]:
mentions = api.mentions_timeline(**query)

Unexpected parameter: screen_name


In [55]:
mentions[0]._json["full_text"]

"Yesterday, I implemented the k-Center attachment strategy. \n\nToday, I'm going to use the simulator developed by @ferencberes91 to compare my MBI algorithm to several other attachment strategies in terms of daily traffic and income."

#### User status is alwys up to date... for past tweets the past user state is hidden...

In [56]:
timeline = api.user_timeline(**query)

In [63]:
timeline[0]._json

{'created_at': 'Sun Dec 05 19:16:42 +0000 2021',
 'id': 1467573664501616647,
 'id_str': '1467573664501616647',
 'full_text': '@benrozemberczki @EdiDataScience @InfAtEd Congratulations! :)',
 'truncated': False,
 'display_text_range': [42, 61],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'benrozemberczki',
    'name': 'Benedek Rozemberczki',
    'id': 1093290008902021120,
    'id_str': '1093290008902021120',
    'indices': [0, 16]},
   {'screen_name': 'EdiDataScience',
    'name': 'CDT in Data Science',
    'id': 2208890396,
    'id_str': '2208890396',
    'indices': [17, 32]},
   {'screen_name': 'InfAtEd',
    'name': 'InformaticsEdinburgh',
    'id': 90873580,
    'id_str': '90873580',
    'indices': [33, 41]}],
  'urls': []},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': 1467466820496068613,
 'in_reply_to_status_id_str': '1467466820496068613',
 'in_reply_to_user_id': 